## Raport: Implementacja Gry Clobber z AI

**Autor:** Jan Powęski
**Data:** 28.05.2025

### 1. Wprowadzenie

Niniejszy raport dotyczy analizy i opisu programu `clobber.py`, który implementuje grę logiczną Clobber oraz agentów sztucznej inteligencji (AI) do rozgrywki. Clobber to gra dla dwóch graczy, rozgrywana na prostokątnej planszy, na której początkowo wszystkie pola są zajęte przez pionki dwóch kolorów. Celem gry jest zbicie wszystkich pionków przeciwnika lub doprowadzenie do sytuacji, w której przeciwnik nie może wykonać ruchu.

Program `clobber.py` implementuje logikę gry, algorytmy przeszukiwania drzewa gry (Minimax i Alfa-Beta) oraz różne funkcje heurystyczne do oceny stanu gry. Umożliwia rozgrywkę w dwóch trybach: "basic" (Minimax vs Minimax z adaptacyjną heurystyką) oraz "extended" (konfigurowalne agenty dla każdego gracza).

### 2. Opis Teoretyczny Metody

Program wykorzystuje klasyczne algorytmy przeszukiwania z teorii gier do podejmowania decyzji przez agentów AI.

#### 2.1. Algorytm Minimax

Minimax to algorytm rekurencyjny używany w teorii gier do wyboru optymalnego ruchu dla gracza, zakładając, że przeciwnik również gra optymalnie. Działa poprzez budowę drzewa gry, gdzie węzły reprezentują stany gry, a krawędzie to możliwe ruchy.
*   **Gracz MAX:** Gracz, dla którego aktualnie szukamy najlepszego ruchu (starający się zmaksymalizować swoją ocenę).
*   **Gracz MIN:** Przeciwnik (starający się zminimalizować ocenę gracza MAX).
Algorytm przeszukuje drzewo do określonej głębokości. W liściach drzewa (lub na osiągniętej głębokości) stosowana jest **funkcja heurystyczna**, która ocenia, jak korzystny jest dany stan gry dla gracza MAX. Wartości te są następnie propagowane w górę drzewa:
*   Węzły MAX wybierają maksymalną wartość spośród swoich dzieci.
*   Węzły MIN wybierają minimalną wartość spośród swoich dzieci.
Ostatecznie, korzeń drzewa otrzymuje wartość, która reprezentuje ocenę najlepszego możliwego do osiągnięcia stanu po wykonaniu optymalnego ruchu, a sam ruch jest również zwracany.

#### 2.2. Algorytm Alfa-Beta (Alpha-Beta Pruning)

Alfa-Beta to optymalizacja algorytmu Minimax. Redukuje liczbę przeszukiwanych węzłów w drzewie gry, odcinając gałęzie, które na pewno nie wpłyną na ostateczną decyzję. Działa poprzez utrzymywanie dwóch wartości podczas przeszukiwania:
*   **Alfa (α):** Najlepsza (najwyższa) wartość znaleziona do tej pory dla gracza MAX na ścieżce do korzenia.
*   **Beta (β):** Najlepsza (najniższa) wartość znaleziona do tej pory dla gracza MIN na ścieżce do korzenia.
Odcięcie następuje, gdy:
*   Dla węzła MIN, jego tymczasowa wartość staje się mniejsza lub równa α ( `wartość <= α` ). Oznacza to, że gracz MAX ma już lepszą alternatywę w innej części drzewa i nie wybierze tej gałęzi.
*   Dla węzła MAX, jego tymczasowa wartość staje się większa lub równa β ( `wartość >= β` ). Oznacza to, że gracz MIN ma już lepszą alternatywę (dla siebie) w innej części drzewa i nie pozwoli graczowi MAX osiągnąć tej gałęzi.
Alfa-Beta daje ten sam wynik co Minimax, ale jest znacznie wydajniejszy, zwłaszcza dla głębokich drzew.

#### 2.3. Funkcje Heurystyczne

Ponieważ pełne przeszukanie drzewa gry Clobber jest obliczeniowo niewykonalne (zbyt duża liczba możliwych stanów), algorytmy przeszukują do ograniczonej głębokości. Funkcje heurystyczne służą do oceny "jakości" stanów gry na tej granicy przeszukiwania. Dobra heurystyka powinna szybko ocenić, który gracz ma przewagę, bez konieczności dalszego rozwijania drzewa. W kodzie zaimplementowano kilka heurystyk, np.:
*   Różnica w liczbie pionków.
*   Różnica w liczbie możliwych ruchów (mobilność).
*   Liczba bezpiecznych pionków.
*   Kontrola centrum planszy.
Implementacja zawiera również mechanizm **heurystyki adaptacyjnej**, która wybiera jedną z predefiniowanych heurystyk w zależności od fazy gry (np. liczby pozostałych pionków na planszy).

### 3. Formalne Sformułowanie Problemu

#### 3.1. Gra Clobber

*   **Gracze:** Dwóch graczy, w kodzie oznaczonych jako `PLAYER_B` ('B') i `PLAYER_W` ('W').
*   **Plansza:** Prostokątna siatka o wymiarach `ROWS` x `COLS` (w kodzie 6x5). Każde pole jest albo puste (`EMPTY` = '_') albo zajęte przez pionek jednego z graczy.
*   **Stan Gry (S):** Reprezentowany przez dwuwymiarową tablicę (listę list w Pythonie) zawierającą konfigurację pionków na planszy.
    `S = board[ROWS][COLS]`, gdzie `board[r][c] ∈ {PLAYER_B, PLAYER_W, EMPTY}`.
*   **Akcje (Ruchy - A(s)):** Gracz może przesunąć swój pionek na sąsiednie (ortogonalnie: góra, dół, lewo, prawo) pole zajęte przez pionek przeciwnika. Pionek przeciwnika jest zdejmowany z planszy (zbity), a pionek gracza zajmuje jego miejsce.
    Formalnie, ruch `m = ((r1, c1), (r2, c2))` jest legalny, jeśli:
    1.  `board[r1][c1]` == `aktualny_gracz`
    2.  `board[r2][c2]` == `przeciwnik(aktualny_gracz)`
    3.  `(r2, c2)` jest sąsiadem ortogonalnym `(r1, c1)`: `|r1-r2| + |c1-c2| = 1`.
*   **Funkcja Przejścia (T(s, a) -> s'):** `apply_move(board, move)` zwraca nowy stan planszy `s'` po wykonaniu ruchu `a` w stanie `s`.
*   **Stan Początkowy (S₀):** Konfiguracja planszy wczytywana z wejścia (np. pliku `board.txt`). W typowej grze Clobber plansza jest na początku całkowicie wypełniona naprzemiennie pionkami graczy.
*   **Warunki Zakończenia Gry (Stany Terminalne):**
    1.  Gracz zbije wszystkie pionki przeciwnika. Ten gracz wygrywa.
    2.  Gracz nie może wykonać żadnego legalnego ruchu (brak pionków, które mogą się ruszyć na pole przeciwnika). Ten gracz przegrywa (przeciwnik wygrywa).
*   **Cel Gry:** Doprowadzić do stanu terminalnego, w którym jest się zwycięzcą.

#### 3.2. Ograniczenia

*   **Rozmiar planszy:** `ROWS = 6`, `COLS = 5`.
*   **Głębokość przeszukiwania (Depth):** `GAME_DEPTH = 3` (domyślnie, może być zmieniona w `AGENT_SETTINGS`). Jest to ograniczenie nałożone na algorytmy Minimax/Alfa-Beta, aby zapewnić rozsądny czas odpowiedzi.
*   **Czas:** Choć nie ma twardego limitu czasu na ruch w kodzie, praktycznie głębokość przeszukiwania i złożoność heurystyk wpływają na czas wykonania.

#### 3.3. Drzewo Decyzyjne (Drzewo Gry)

*   **Korzeń:** Aktualny stan gry.
*   **Węzły:** Możliwe stany gry.
*   **Krawędzie:** Legalne ruchy prowadzące z jednego stanu do drugiego.
*   **Poziomy:** Na przemian reprezentują tury gracza MAX (np. 'B') i MIN (np. 'W').
*   **Liście:**
    *   Stany terminalne (zwycięstwo, porażka).
    *   Stany osiągnięte na maksymalnej głębokości przeszukiwania. Wartość tych liści jest szacowana przez funkcję heurystyczną.

Celem agenta AI jest znalezienie ścieżki od korzenia do liścia (w ramach ustalonej głębokości), która maksymalizuje jego funkcję użyteczności (heurystykę), zakładając, że przeciwnik będzie minimalizował tę samą funkcję użyteczności.

### 4. Opis Idei Rozwiązania (Implementacja w `clobber.py`)

Program implementuje grę Clobber i agentów AI w następujący sposób:

1.  **Reprezentacja Planszy i Podstawowe Operacje:**
    *   Plansza jest listą list (`board`).
    *   Stałe `PLAYER_B`, `PLAYER_W`, `EMPTY` definiują zawartość pól.
    *   `DIRECTIONS` ułatwia generowanie sąsiednich pól.
    *   `read_board()` wczytuje stan początkowy.
    *   `print_board_to_stdout()` wyświetla planszę.
    *   `get_opponent()` zwraca symbol przeciwnika.

2.  **Logika Gry:**
    *   `generate_moves(board, player)`: Kluczowa funkcja, która dla danego gracza i stanu planszy generuje listę wszystkich możliwych legalnych ruchów. Iteruje po wszystkich polach, sprawdza, czy należy do gracza, a następnie sprawdza sąsiednie pola w poszukiwaniu pionków przeciwnika.
    *   `apply_move(board, move)`: Tworzy kopię planszy (aby nie modyfikować oryginalnej podczas przeszukiwania) i wykonuje na niej ruch, zmieniając `board[r1][c1]` na `EMPTY` i `board[r2][c2]` na pionek gracza, który wykonał ruch.

3.  **Funkcje Heurystyczne:**
    *   Zdefiniowano zestaw funkcji heurystycznych (`heuristic_B_1`, `heuristic_B_2`, ..., `heuristic_W_3`). Każda z nich przyjmuje `board` i `player_perspective` (gracza, z perspektywy którego oceniamy stan) i zwraca wartość liczbową.
        *   `heuristic_B_1`: Różnica w liczbie pionków. Prosta, ale często skuteczna.
        *   `heuristic_B_2`: Różnica w mobilności (liczbie możliwych ruchów).
        *   `heuristic_B_3`: Liczba własnych ruchów.
        *   `heuristic_W_1`: Liczba "bezpiecznych" pionków (niezagrożonych bezpośrednim zbiciem).
        *   `heuristic_W_2`: Ujemna liczba ruchów przeciwnika (im mniej ruchów ma przeciwnik, tym lepiej).
        *   `heuristic_W_3`: Kontrola centralnych pól planszy.
    *   `HEURISTICS`: Słownik mapujący gracza i klucz heurystyki na odpowiednią funkcję.
    *   `choose_adaptive_heuristic(board, player)`: Wybiera heurystykę ('1', '2' lub '3') na podstawie liczby pionków na planszy, próbując dostosować strategię do fazy gry (początkowa, środkowa, końcowa).

4.  **Algorytmy Przeszukiwania:**
    *   `minimax(board, depth, maximizing_turn, root_player, heuristic_for_root_player)`:
        *   Implementuje algorytm Minimax.
        *   `depth`: Aktualna głębokość rekurencji.
        *   `maximizing_turn`: `True`, jeśli to tura gracza MAX, `False` dla MIN.
        *   `root_player`: Gracz, dla którego pierwotnie wywołano funkcję (z perspektywy którego liczymy heurystykę).
        *   `heuristic_for_root_player`: Funkcja heurystyczna do użycia.
        *   Rekurencyjnie wywołuje siebie dla każdego możliwego ruchu, zmniejszając `depth` i zmieniając `maximizing_turn`.
        *   Zwraca `(ocena, najlepszy_ruch, liczba_odwiedzonych_węzłów)`.
    *   `alphabeta(board, depth, alpha, beta, maximizing_turn, root_player, heuristic_for_root_player)`:
        *   Implementuje algorytm Alfa-Beta.
        *   `alpha`, `beta`: Wartości dla odcięć.
        *   Działa podobnie do Minimax, ale przekazuje i aktualizuje `alpha` i `beta`, dokonując odcięć, gdy `beta <= alpha`.

5.  **Struktura Rozgrywki:**
    *   **Tryb "basic" (`play_basic_minimax_vs_minimax`):**
        *   Obaj gracze używają algorytmu Minimax.
        *   Heurystyka jest wybierana adaptacyjnie (`choose_adaptive_heuristic`) dla każdego gracza w jego turze.
        *   Gra toczy się do momentu, gdy jeden z graczy nie ma ruchów lub nie ma pionków.
    *   **Tryb "extended" (`play_extended_agents_game`):**
        *   Wykorzystuje klasę `Agent`. Każdy agent (`agent_B`, `agent_W`) jest konfigurowany przy starcie (algorytm, klucz heurystyki, głębokość) za pomocą słowników `AGENT_B_SETTINGS` i `AGENT_W_SETTINGS`.
        *   Agent posiada metodę `get_move_and_stats(board)`, która:
            1.  Wybiera funkcję heurystyczną (może być "adaptive" lub stała).
            2.  Wywołuje odpowiedni algorytm (Minimax lub Alfa-Beta).
            3.  Zwraca `(ocena, ruch, liczba_węzłów, czas_tura)`.
        *   Pętla gry zarządza turami, wywołując agentów, aktualizując planszę i sprawdzając warunki końca gry.
    *   `main()`: Odczytuje planszę, a następnie na podstawie `GAME_VERSION_CHOICE` uruchamia odpowiedni tryb gry.

6.  **Zbieranie Statystyk:**
    *   Oba tryby gry zliczają całkowitą liczbę odwiedzonych węzłów w drzewie gry oraz całkowity czas poświęcony na obliczenia przez AI. Te informacje są wypisywane na `sys.stderr`.

### 5. Krótki Przykład w Jupyter Notebook

Poniżej znajduje się fragment kodu, który można uruchomić w Jupyter Notebook, aby zademonstrować działanie podstawowych funkcji i wykonanie jednego ruchu przez agenta AI. Dla uproszczenia, kluczowe funkcje i stałe zostały skopiowane.

In [2]:
# Komórka 1: Definicje i funkcje (skopiowane/zaadaptowane z clobber.py)
import copy
import time

# --- Constants ---
PLAYER_B = 'B'
PLAYER_W = 'W'
EMPTY = '_'
DIRECTIONS = [(1, 0), (-1, 0), (0, 1), (0, -1)]
# Użyjemy mniejszej planszy i głębokości dla przykładu
COLS, ROWS = 3, 2 
GAME_DEPTH_EXAMPLE = 2

# --- Board Utilities ---
def print_board(board):
    for row in board:
        print(' '.join(row))

def get_opponent(player):
    return PLAYER_W if player == PLAYER_B else PLAYER_B

def generate_moves(board, player, current_rows, current_cols):
    moves = []
    for r in range(current_rows):
        for c in range(current_cols):
            if board[r][c] == player:
                for dr, dc in DIRECTIONS:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < current_rows and 0 <= nc < current_cols and board[nr][nc] == get_opponent(player):
                        moves.append(((r, c), (nr, nc)))
    return moves

def apply_move(board, move):
    (r1, c1), (r2, c2) = move
    new_board = copy.deepcopy(board)
    new_board[r2][c2] = new_board[r1][c1]
    new_board[r1][c1] = EMPTY
    return new_board

# --- Heuristic Example (B_2 from the code) ---
def heuristic_example_mobility(board, player_perspective, current_rows, current_cols):
    opp = get_opponent(player_perspective)
    # Pass current_rows and current_cols to generate_moves
    return len(generate_moves(board, player_perspective, current_rows, current_cols)) - \
           len(generate_moves(board, opp, current_rows, current_cols))

# --- Search Algorithm Example (Alpha-Beta) ---
def alphabeta(board, depth, alpha, beta, maximizing_turn, root_player, heuristic_fn, current_rows, current_cols):
    nodes_visited_total = 1
    actual_mover = root_player if maximizing_turn else get_opponent(root_player)
    moves = generate_moves(board, actual_mover, current_rows, current_cols)

    if depth == 0 or not moves:
        return heuristic_fn(board, root_player, current_rows, current_cols), None, nodes_visited_total

    best_move = moves[0] if moves else None

    if maximizing_turn:
        max_eval = float('-inf')
        for move in moves:
            new_board = apply_move(board, move)
            eval_val, _, nodes_subtree = alphabeta(new_board, depth - 1, alpha, beta, False, root_player, heuristic_fn, current_rows, current_cols)
            nodes_visited_total += nodes_subtree
            if eval_val > max_eval:
                max_eval = eval_val
                best_move = move
            alpha = max(alpha, eval_val)
            if beta <= alpha:
                break
        return max_eval, best_move, nodes_visited_total
    else:
        min_eval = float('inf')
        for move in moves:
            new_board = apply_move(board, move)
            eval_val, _, nodes_subtree = alphabeta(new_board, depth - 1, alpha, beta, True, root_player, heuristic_fn, current_rows, current_cols)
            nodes_visited_total += nodes_subtree
            if eval_val < min_eval:
                min_eval = eval_val
                best_move = move
            beta = min(beta, eval_val)
            if beta <= alpha:
                break
        return min_eval, best_move, nodes_visited_total

print("Definicje wczytane.")


Definicje wczytane.


In [3]:
# Komórka 2: Przygotowanie i uruchomienie przykładu

# Prosta plansza 2x3
# B W B
# W B W
initial_board_example = [
    [PLAYER_B, PLAYER_W, PLAYER_B],
    [PLAYER_W, PLAYER_B, PLAYER_W]
]
CURRENT_ROWS, CURRENT_COLS = 2, 3 # Zgodnie z initial_board_example

print("Plansza początkowa:")
print_board(initial_board_example)
print(f"\nRozmiar planszy: ROWS={CURRENT_ROWS}, COLS={CURRENT_COLS}")

# Kto gra?
current_player_example = PLAYER_B
print(f"\nGracz wykonujący ruch: {current_player_example}")

# Wygenerujmy możliwe ruchy dla gracza B
possible_moves_b = generate_moves(initial_board_example, current_player_example, CURRENT_ROWS, CURRENT_COLS)
print(f"Możliwe ruchy dla {current_player_example}: {possible_moves_b}")

# Użyjmy Alpha-Beta do znalezienia najlepszego ruchu dla gracza B
# z głębokością GAME_DEPTH_EXAMPLE i heurystyką heuristic_example_mobility
start_time = time.perf_counter()
eval_score, best_move, nodes = alphabeta(
    initial_board_example, 
    GAME_DEPTH_EXAMPLE, 
    float('-inf'), 
    float('inf'), 
    True,  # Maximizing turn
    current_player_example,
    heuristic_example_mobility,
    CURRENT_ROWS, # Przekazanie aktualnych wymiarów
    CURRENT_COLS  # Przekazanie aktualnych wymiarów
)
end_time = time.perf_counter()

print(f"\n--- Wynik działania Alpha-Beta (głębokość: {GAME_DEPTH_EXAMPLE}) ---")
print(f"Najlepszy ruch dla {current_player_example}: {best_move}")
print(f"Ocena heurystyczna tego ruchu: {eval_score}")
print(f"Liczba odwiedzonych węzłów: {nodes}")
print(f"Czas obliczeń: {end_time - start_time:.4f}s")

if best_move:
    print("\nPlansza po wykonaniu najlepszego ruchu:")
    final_board_example = apply_move(initial_board_example, best_move)
    print_board(final_board_example)
else:
    print("\nBrak możliwych ruchów!")

Plansza początkowa:
B W B
W B W

Rozmiar planszy: ROWS=2, COLS=3

Gracz wykonujący ruch: B
Możliwe ruchy dla B: [((0, 0), (1, 0)), ((0, 0), (0, 1)), ((0, 2), (1, 2)), ((0, 2), (0, 1)), ((1, 1), (0, 1)), ((1, 1), (1, 2)), ((1, 1), (1, 0))]

--- Wynik działania Alpha-Beta (głębokość: 2) ---
Najlepszy ruch dla B: ((0, 0), (1, 0))
Ocena heurystyczna tego ruchu: 0
Liczba odwiedzonych węzłów: 18
Czas obliczeń: 0.0003s

Plansza po wykonaniu najlepszego ruchu:
_ W B
B B W


**Wyjaśnienie przykładu z Jupyter Notebook:**
1.  W pierwszej komórce definiujemy niezbędne stałe i funkcje skopiowane z `clobber.py`. Zmieniono `ROWS` i `COLS` na mniejsze dla szybszego działania przykładu oraz `GAME_DEPTH_EXAMPLE` również na małą wartość. Funkcje `generate_moves` i `heuristic_example_mobility` (oraz `alphabeta`) zostały dostosowane, aby przyjmować `current_rows` i `current_cols` jako argumenty, ponieważ globalne `ROWS` i `COLS` mogą być inne.
2.  W drugiej komórce:
    *   Tworzymy małą, przykładową planszę `initial_board_example` (2x3).
    *   Określamy, który gracz (`current_player_example`) ma wykonać ruch.
    *   Wyświetlamy możliwe ruchy dla tego gracza.
    *   Wywołujemy funkcję `alphabeta` z tą planszą, graczem, zadaną głębokością i przykładową heurystyką (`heuristic_example_mobility`).
    *   Drukujemy wynik: najlepszy znaleziony ruch, jego ocenę, liczbę odwiedzonych węzłów i czas obliczeń.
    *   Jeśli znaleziono ruch, stosujemy go i drukujemy planszę wynikową.

Ten przykład ilustruje kluczowy krok podejmowania decyzji przez agenta AI: ocenę stanu, przeszukanie drzewa gry do pewnej głębokości i wybór ruchu prowadzącego do najlepszego (według heurystyki) stanu.

### 6. Podsumowanie i Wnioski

Program `clobber.py` stanowi solidną implementację gry Clobber z agentami AI opartymi na algorytmach Minimax i Alfa-Beta. Zastosowanie różnych heurystyk, w tym mechanizmu adaptacyjnego, pozwala na badanie różnych strategii gry. Kod jest dobrze zorganizowany, z wyraźnym podziałem na logikę gry, algorytmy AI i funkcje pomocnicze.

Możliwe kierunki dalszego rozwoju mogłyby obejmować:
*   Implementację bardziej zaawansowanych heurystyk.
*   Dodanie iteracyjnego pogłębiania (Iterative Deepening) dla lepszego zarządzania czasem.
*   Stworzenie interfejsu graficznego użytkownika.
*   Optymalizację generowania ruchów lub reprezentacji planszy dla większej wydajności.

Kod dostarcza wartościowej bazy do nauki i eksperymentowania z algorytmami sztucznej inteligencji w kontekście gier planszowych.

---